In [ ]:
# Cell 1: Initial setup and data loading
from itertools import compress
import matplotlib.pyplot as plt
import numpy as np
import os
import mne
import mne_nirs
from mne_nirs.statistics import run_glm
import statsmodels.formula.api as smf
import pandas as pd

# Set up FreeSurfer paths
# wsl_freesurfer_home = r"\\wsl$\\Ubuntu-22.04\\usr\\local\\freesurfer\\7.4.1"
# wsl_subjects_dir = wsl_freesurfer_home + r"\\subjects"
# subjects_dir = wsl_subjects_dir
# os.environ['FREESURFER_HOME'] = wsl_freesurfer_home
# os.environ['SUBJECTS_DIR'] = wsl_subjects_dir
# print("FreeSurfer environment configured for WSL")

# Function to extract trial treatment from filename
def extract_trial_treatment(snirf_file):
    filename = os.path.basename(snirf_file)
    filename_parts = filename.replace('.snirf', '').split('_')
    
    return filename_parts[1]  # Assuming treatment is always the second part

# Function to extract participant ID from filename
def extract_participant_id(snirf_file):
    filename = os.path.basename(snirf_file)
    filename_parts = filename.replace('.snirf', '').split('_')
    
    return filename_parts[2]  # Assuming participant ID is always the third part

def extract_trial_name(snirf_file):
    filename = os.path.basename(snirf_file)
    filename_parts = filename.replace('.snirf', '').split('_')
    
    return f"{filename_parts[1]}_{filename_parts[2]}"  # treatment_participantID

def raw_intensity_to_hemo(snirf_file, participant_id):
    """Process a single SNIRF file through the complete pipeline"""
    
    trial_name = extract_trial_name(snirf_file)
    print(f"Loading data from: {snirf_file} (Trial: {trial_name})")
    
    # Load and process data (following your original cells 1-12)
    raw_intensity = mne.io.read_raw_snirf(snirf_file, optode_frame='unknown', verbose="info")
    raw_intensity.load_data()

    # Resample
    print(f"Data shape: {raw_intensity.get_data().shape}")
    print(f"Sampling frequency: {raw_intensity.info['sfreq']} Hz")
    raw_intensity.resample(5)
    
    # Convert to hemoglobin
    raw_od = mne.preprocessing.nirs.optical_density(raw_intensity, verbose=False)
    raw_od_corrected = mne.preprocessing.nirs.temporal_derivative_distribution_repair(raw_od, verbose=False)
    raw_od_filtered = raw_od_corrected.copy().filter(0.01, 0.25, method='fir', fir_design='firwin', verbose=False)
    raw_hemo = mne.preprocessing.nirs.beer_lambert_law(raw_od_filtered, ppf=0.1)
    
    return raw_hemo, trial_name

# Define your two files
snirf_file1 = "..\\data\\raw\\250624_1014_20250624_Keehn_Logan_NS_Keehn_Logan_concat.snirf"
snirf_file2 = "..\\data\\raw\\250625_1408_20250625_Rudd_Will_NS_rudd_will.snirf"  # Replace with your second file path

# Process both files
print("=== Processing File 1 ===")
raw_hemo1, trial1 = raw_intensity_to_hemo(snirf_file1, 1)

print("\n=== Processing File 2 ===")
raw_hemo2, trial2 = raw_intensity_to_hemo(snirf_file2, 2)

FreeSurfer environment configured for WSL
=== Processing File 1 ===
Loading data from: ..\data\raw\250624_1014_20250624_Keehn_Logan_NS_Keehn_Logan_concat.snirf (Trial: MRSA)
Loading c:\Users\jdtull\Documents\CRED\Projects\Design Cognition\Design-Cognition\cred_fnirs_toolkit\notebooks\..\data\raw\250624_1014_20250624_Keehn_Logan_NS_Keehn_Logan_concat.snirf
Found jitter of 0.000000% in sample times.
Reading 0 ... 19631  =      0.000 ...  2453.875 secs...


C:\Users\jdtull\AppData\Local\Temp\ipykernel_35064\377737358.py:51: RuntimeWarning: Negative intensities encountered. Setting to abs(x)
  raw_od = mne.preprocessing.nirs.optical_density(raw_intensity, verbose=False)


Data shape: (408, 19632)
Sampling frequency: 8.0 Hz
Design matrix shape: (9816, 128)
Design matrix columns: ['Design problem context', 'Generate stakeholder needs', 'Prioritize stakeholder needs', 'Stakeholder identification', 'Stakeholder personas', 'drift_1', 'drift_2', 'drift_3', 'drift_4', 'drift_5', 'drift_6', 'drift_7', 'drift_8', 'drift_9', 'drift_10', 'drift_11', 'drift_12', 'drift_13', 'drift_14', 'drift_15', 'drift_16', 'drift_17', 'drift_18', 'drift_19', 'drift_20', 'drift_21', 'drift_22', 'drift_23', 'drift_24', 'drift_25', 'drift_26', 'drift_27', 'drift_28', 'drift_29', 'drift_30', 'drift_31', 'drift_32', 'drift_33', 'drift_34', 'drift_35', 'drift_36', 'drift_37', 'drift_38', 'drift_39', 'drift_40', 'drift_41', 'drift_42', 'drift_43', 'drift_44', 'drift_45', 'drift_46', 'drift_47', 'drift_48', 'drift_49', 'drift_50', 'drift_51', 'drift_52', 'drift_53', 'drift_54', 'drift_55', 'drift_56', 'drift_57', 'drift_58', 'drift_59', 'drift_60', 'drift_61', 'drift_62', 'drift_63', 'd

C:\Users\jdtull\AppData\Local\Temp\ipykernel_35064\377737358.py:44: RuntimeWarning: Extraction of measurement date from SNIRF file failed. The date is being set to January 1st, 2000, instead of 2025-06-25
  raw_intensity = mne.io.read_raw_snirf(snirf_file, optode_frame='unknown', verbose="info")
C:\Users\jdtull\AppData\Local\Temp\ipykernel_35064\377737358.py:51: RuntimeWarning: Negative intensities encountered. Setting to abs(x)
  raw_od = mne.preprocessing.nirs.optical_density(raw_intensity, verbose=False)


Data shape: (408, 17541)
Sampling frequency: 8.138020833333334 Hz
Design matrix shape: (8622, 115)
Design matrix columns: ['Design problem context', 'Generate stakeholder needs', 'Prioritize stakeholder needs', 'Stakeholder identification', 'Thinkaloud practice AUT', 'Thinkaloud practice basic math', 'Thinkalound practice anagram', 'drift_1', 'drift_2', 'drift_3', 'drift_4', 'drift_5', 'drift_6', 'drift_7', 'drift_8', 'drift_9', 'drift_10', 'drift_11', 'drift_12', 'drift_13', 'drift_14', 'drift_15', 'drift_16', 'drift_17', 'drift_18', 'drift_19', 'drift_20', 'drift_21', 'drift_22', 'drift_23', 'drift_24', 'drift_25', 'drift_26', 'drift_27', 'drift_28', 'drift_29', 'drift_30', 'drift_31', 'drift_32', 'drift_33', 'drift_34', 'drift_35', 'drift_36', 'drift_37', 'drift_38', 'drift_39', 'drift_40', 'drift_41', 'drift_42', 'drift_43', 'drift_44', 'drift_45', 'drift_46', 'drift_47', 'drift_48', 'drift_49', 'drift_50', 'drift_51', 'drift_52', 'drift_53', 'drift_54', 'drift_55', 'drift_56', 'dr